In [1]:
using Pkg
Pkg.activate("dp", shared=true)

  Activating project at `C:\Users\luked\.julia\environments\dp`


In [2]:
using DynamicalSystems
using OrdinaryDiffEq
using GLMakie
using DataStructures: CircularBuffer

In [3]:
#o = Observable(1)
#
#l1 = on(o) do val
#    println("Observable now has a value $val")
#end
#
#o[] = 2

In [4]:
const L1 = 1.0
const L2 = 0.9
M = 2
u0 = [π/3, 0, 3π/4, -2]
dp = Systems.double_pendulum(u0; L1, L2)

4-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  doublependulum_rule
 ODE solver:    Tsit5
 ODE kwargs:    (abstol = 1.0e-6, reltol = 1.0e-6)
 parameters:    [10.0, 1.0, 0.9, 1.0, 1.0]
 time:          0.0
 state:         [1.0471975511965976, 0.0, 2.356194490192345, -2.0]


In [5]:
diffeq = (alg=Tsit5(), adaptive=false, dt=0.005)

integ = dp.integ

t: 0.0
u: 4-element SVector{4, Float64} with indices SOneTo(4):
  1.0471975511965976
  0.0
  2.356194490192345
 -2.0

In [8]:
function xycoords(state)
    θ1 = state[1]
    θ2 = state[3]
    x1 = L1 * sin(θ1)
    y1 = -L1 * cos(θ1)
    x2 = x1 + L2 * sin(θ2)
    y2 = y1 - L2 * cos(θ2)

    return x1, x2, y1, y2
end

xycoords (generic function with 1 method)

In [9]:
function progress_for_one_step!(integ)
    step!(integ)
    
    return xycoords(integ)
end

progress_for_one_step! (generic function with 1 method)

In [10]:
x1, x2, y1, y2 = xycoords(u0)
rod = Observable([Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)])
balls = Observable([Point2f(x1, y1), Point2f(x2, y2)])

Observable(Point{2, Float32}[[0.8660254, -0.5], [1.5024215, 0.13639611]])


In [11]:
tail = 300

traj = CircularBuffer{Point2f}(tail)
fill!(traj, Point2f(x2, y2))
traj = Observable(traj)

Observable(Point{2, Float32}[[1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611]  …  [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611]])


In [36]:
fig = Figure(); display(fig)
ax = Axis(fig[1,1])

Axis with 0 plots:


In [13]:
lines!(ax, rod; linewidth=4, color=:purple)
scatter!(ax, balls; marker=:circle, strokewidth=2, strokecolor=:purple, color=:black, markersize=[8,12])

Scatter{Tuple{Vector{Point{2, Float32}}}}

In [14]:
c = to_color(:purple)
tailcol=[RGBAf(c.r, c.g, c.b, (i/tail)^2) for i in 1:tail]
lines!(ax, traj; linewidth=3, color=tailcol)

Lines{Tuple{CircularBuffer{Point{2, Float32}}}}

In [35]:
ax.title = "Double Pendulum"
ax.aspect = DataAspect()
l = 1.05(L1+L2)
xlims!(ax, -l, l)
ylims!(ax, -l, l)

In [17]:
function animstep!(integ, rod, balls, traj)
    x1, x2, y1, y2 = progress_for_one_step!(integ)
    rod[] = [Point2f(0, 0), Point2f(x1,x1), Point2f(x2,y2)]
    balls[] = [Point2f(x1,x1), Point2f(x2,y2)]
    push!(traj[], Point2f(x2,y2))
    traj[] = traj[]
end

animstep! (generic function with 1 method)

In [18]:
#for i in 1:1000
#    animstep!(integ, rod, balls, traj)
#    sleep(0.001)
#end

In [37]:
function makefig(u0)
    dp = Systems.double_pendulum(u0; L1, L2)
    integ = dp.integ
    x1,x2,y1,y2 = xycoords(u0)
    rod   = Observable([Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)])
    balls = Observable([Point2f(x1, y1), Point2f(x2, y2)])
    traj = CircularBuffer{Point2f}(tail)
    fill!(traj, Point2f(x2, y2)) # add correct values to the circular buffer
    traj = Observable(traj) # make it an observable
    fig = Figure(); display(fig)
    ax = Axis(fig[1,1])
    lines!(ax, rod; linewidth = 4, color = :purple)
    scatter!(ax, balls; marker = :circle, strokewidth = 2, 
        strokecolor = :purple,
        color = :black, markersize = [8, 12]
    )
    lines!(ax, traj; linewidth = 3, color = tailcol)
    ax.title = "Double Pendulum"
    ax.aspect = DataAspect()
    l = 1.05(L1+L2)
    xlims!(ax, -l, l)
    ylims!(ax, -l, l)
    # also return the figure object, we'll use it!
    return fig, integ, rod, balls, traj
end

makefig (generic function with 1 method)

In [20]:
fig, integ, rod, balls, traj = makefig(u0)
frames = 1:200
record(fig, "dp_vid.mp4", frames; framerate=60) do i
    for j in 1:5
        animstep!(integ, rod, balls, traj)
    end
end

"dp_vid.mp4"

In [39]:
fig, integ, rod, balls, traj = makefig(u0)

run = Button(fig[2,1]; label="Run", tellwidth=false)

isrunning = Observable(false)
on(run.clicks) do clicks; isrunning[] = !isrunning[]; end
on(run.clicks) do clicks
    @async while isrunning[]
        isopen(fig.scene) || break
        animstep!(integ, rod, balls, traj)
        sleep(0.001)
    end
end

ax = content(fig[1,1])
Makie.deactivate_interaction!(ax, :rectanglezoom)
spoint = select_point(ax.scene, marker=:circle)

Observable(Float32[0.0, 0.0])


In [38]:
function θωcoords(x, y)
    θ = atan(y,x) + π/2

    return SVector(θ,0,0,0)
end

on(spoint) do z
    x, y = z
    u = θωcoords(x,y)
    reinit!(integ,u)

    x1,x2,y1,y2=xycoords(u)
    traj[] .= fill(Point2f(x2,y2), length(traj[]))
    traj[] = traj[]
    rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
    balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
end

ObserverFunction defined at d:\GitHub\julia-l-and-d\double_pendulum\dp.ipynb:8 operating on Observable(Float32[0.0, 0.0])